# Assignment 3: Mail Services
Create a messaging application, which sends notification letters to a group of people at a specified time, under the following conditions:  
The names and mail addresses of the group members are stored in either xml or json file.  
The content of the letter is stored in a text file.  
The letter should start with the salutation “Dear XX NN,”, where
- XX is either ‘Mr’ or ‘Ms’, depending on the gender of the receiver;
- NN is the name of the receiver.
The gender of the receiver can be discovered by use of a web service, hosted at http://www.thomas-bayer.com/restnames/ 


1. Read json file with names and mails
2. Read text file with content of the email
3. Determine gender
4. Start text with Dear XX NN
5. Send mail

In [1]:
import requests
import json
import xmltodict

from pprint import pprint as pp
from flask import Flask, render_template, request, g
from flask_cors import CORS, cross_origin
from apscheduler.schedulers.background import BackgroundScheduler
import atexit
import smtplib

In [2]:
def get_salutation(name:str) -> str:
    gender_url = f'http://www.thomas-bayer.com/restnames/name.groovy?name='
    data = requests.get(gender_url + name)
    xpars = xmltodict.parse(data.text)
    male = xpars['restnames']['nameinfo']['male'] == 'true'
    female = xpars['restnames']['nameinfo']['female'] == 'true'

    if male and female:
        prefix = 'Mx'
    elif male:
        prefix = 'Mr'
    elif female:
        prefix = 'Ms'

    return f'Dear {prefix}'

def read_json(path:str) -> dict:
    with open(path, 'r') as file:
        data = file.read()
        data_dict = json.loads(data)
        return data_dict
        
def read_content(path:str) -> str:
    with open(path, 'r') as file:
        content = file.read()
        return content

In [3]:
def send_mail(receiver: str, subject: str, message: str):
    print("receiver:",receiver)
    print("subject:",subject)
    print("message:",message)
    print("----")
    
def send_mails():
    json_path = 'members.json'
    content_path = 'content.txt'
    content = read_content(content_path)
    members = read_json(json_path)
    for name, mail in members.items():
        fname, *lname = name.split(' ')
        salutation = f'{get_salutation(fname)} {name}'
        msg = f'{salutation}\n{content}'
        send_mail(mail, "Subject here", msg)

In [4]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

########################
#####     CRON     #####
########################
scheduler = BackgroundScheduler()
scheduler.add_job(func=send_mails, trigger="interval", seconds=30) 
scheduler.start()
atexit.register(lambda: scheduler.shutdown())
#######################

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


receiver: test@testesen.dk
subject: Subject here
message: Dear Mr Nikolaj Thorsen Nielsen
Hello World
----
receiver: awesome@mail.com
subject: Subject here
message: Dear Mr Daniel Lindholm
Hello World
----
receiver: another@mail.com
subject: Subject here
message: Dear Mr Michael Ejdal Lundsgaard
Hello World
----
receiver: jacob@b.org
subject: Subject here
message: Dear Mr Jacob Borg
Hello World
----
receiver: test@testesen.dk
subject: Subject here
message: Dear Mr Nikolaj Thorsen Nielsen
Hello World
----
receiver: awesome@mail.com
subject: Subject here
message: Dear Mr Daniel Lindholm
Hello World
----
receiver: another@mail.com
subject: Subject here
message: Dear Mr Michael Ejdal Lundsgaard
Hello World
----
receiver: jacob@b.org
subject: Subject here
message: Dear Mr Jacob Borg
Hello World
----
receiver: test@testesen.dk
subject: Subject here
message: Dear Mr Nikolaj Thorsen Nielsen
Hello World
----
receiver: awesome@mail.com
subject: Subject here
message: Dear Mr Daniel Lindholm
Hello